# Introduction


# Prerequisites


In [123]:
!pip install langchain | tail -n 1
!pip install langchain-ibm | tail -n 1
!pip install langchain-community | tail -n 1
!pip install ibm-watsonx-ai | tail -n 1
# !pip install ibm_watson_machine_learning | tail -n 1
!pip install chromadb | tail -n 1
!pip install tiktoken | tail -n 1
!pip install python-dotenv | tail -n 1
!pip install bs4 | tail -n 1

python(51910) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(51914) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(51923) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(51927) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(51931) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(51940) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(51943) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(51947) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [124]:
import os
import json

from dotenv import load_dotenv

from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough

# from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models import ModelInference
# from ibm_watsonx_ai.foundation_models.schema import TextChatParameters
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

In [125]:
load_dotenv()

WATSONX_API_KEY = os.getenv("WATSONX_API_KEY")
WATSONX_URL=os.getenv("WATSONX_URL")
WATSONX_PROJECT_ID=os.getenv("WATSONX_PROJECT_ID")

# Initialization a basic agent with no tools
The model parameters available can be found [here](https://ibm.github.io/watson-machine-learning-sdk/model.html). We experimented with various model parameters, including temperature, minimum and maximum new tokens and stop sequences. Learn more about model parameters and what they mean in the [watsonx docs](https://www.ibm.com/docs/en/watsonx/saas).

In [126]:
model_id="ibm/granite-3-8b-instruct"

llm = WatsonxLLM(
  model_id=model_id,
  url=WATSONX_URL,
  apikey=WATSONX_API_KEY,
  project_id=WATSONX_PROJECT_ID,
  params={
    'DECODING_METHOD': "greedy",
    'TEMPERATURE': 0,
    'MIN_NEW_TOKENS': 5,
    'MAX_NEW_TOKENS': 250,
    'STOP_SEQUENCES': ["Human:", "Observation"],
    },
)

In [127]:
template = "Answer the {query} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)

Set up a chain with our prompt and our LLM. This allows the generative model to produce a response.

In [128]:
agent = prompt | llm

Test Agent responds to a basic query.

In [129]:
# agent responds to a basic query.
print(agent.invoke({"query": 'What is New Zealand strategic goods list?'}))

 Do not try to make up an answer.

The New Zealand Strategic Goods List is a document that identifies goods, technologies, and services that are subject to export licensing controls due to their potential impact on national security, foreign policy, or law enforcement. The list is maintained by the New Zealand Ministry of Foreign Affairs and Trade and is used to regulate the export of sensitive items to ensure they are not used for activities that could harm New Zealand's interests or international obligations. The list includes items such as military equipment, dual-use goods (items with both civilian and military applications), and certain services related to defense and security.

The New Zealand Strategic Goods List is not publicly available, and the specific items listed are considered sensitive and confidential. Therefore, I cannot provide a detailed list of the items included in the New Zealand Strategic Goods List. However, the general categories of items covered by the list in

In [130]:
# Cannot answer a question about a specific category, as it has no context.
print(agent.invoke({"query": 'What is category 9 of New Zealand strategic goods list?'}))

 Do not make up an answer.

Category 9 of New Zealand's Strategic Goods List includes goods that are used in the production of goods in Category 8, which are considered to be of strategic importance. This category primarily consists of raw materials, components, and sub-assemblies. However, the specific items listed under Category 9 are not publicly available without access to the official New Zealand Strategic Goods List. Therefore, I cannot provide the exact items included in Category 9.


# 3. Setp up RAG with vector DB
### 3.1 Establish the knowledge base and retriever
#### Prepared Documents

Text is extracted to approriated files before by script in `RAG/extractor/py`.

Now, we just load them.

In [131]:
dir_path = f'{os.getcwd()}/RAG/parsed/'

filenames = [
  "_1_terms_14_32",
  "_2_acronyms_and_abbreviations_33_36",
  "_3_category_9_aerospace_and_propulsion_286_308",
]

with open(f'{dir_path}{filenames[0]}.json', 'r') as f:
  terms_json = json.load(f)
with open(f'{dir_path}{filenames[1]}.json', 'r') as f:
  abbreviations_json = json.load(f)
with open(f'{dir_path}{filenames[2]}.json', 'r') as f:
  items_json = json.load(f)
  
with open(f'{dir_path}{filenames[0]}.txt', 'r') as f:
  terms_txt = f.read()
with open(f'{dir_path}{filenames[1]}.txt', 'r') as f:
  abbreviations_txt = f.read()
with open(f'{dir_path}{filenames[2]}.txt', 'r') as f:
  cat9_items_txt = f.read()

#### Embedding
The embedding model that we are using is an [IBM Slate](https://ibm.github.io/watsonx-ai-python-sdk/fm_embeddings.html) model through the watsonx.ai embeddings service. Let's initialize it.

In [132]:
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=WATSONX_URL,
    apikey=WATSONX_API_KEY,
    project_id=WATSONX_PROJECT_ID,
)

#### Store data to vector DB

In [133]:
def create_vectorstore_from_text(text, collection_name):
    """Create a vector store from text."""
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
    doc_splits = text_splitter.create_documents([text])
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        collection_name=collection_name,
        embedding=embeddings,
    )
    return vectorstore

terms_vectorstore = create_vectorstore_from_text(terms_txt, "terms")
abbreviations_vectorstore = create_vectorstore_from_text(abbreviations_txt, "abbreviations")
cat9_items_vectorstore = create_vectorstore_from_text(cat9_items_txt, "items")

#### Create retriever

In [134]:
cat9_items_retriever = cat9_items_vectorstore.as_retriever()

### 3.2 Define the agent's RAG tool

Define the tools (functions) our agent will be using. This tool's only parameter is the user query. The tool description is also noted to inform the agent of the use of the tool. This way, the agent knows when to call this tool. This tool can be used by the agentic RAG system for routing the user query to the vector store if it pertains to NZSGL.

In [135]:
@tool
def get_NZSGL_cat9_items_context(question: str):
  """
  Retrieve context for a question about New Zealand Strategic Goods List category 9 items.
  This function uses a vector store to find relevant context for questions about category 9 items of the New Zealand strategic goods list.
  """
  context = cat9_items_retriever.invoke(question)
  return context

tools = [get_NZSGL_cat9_items_context]

# 4. Prompting

### 4.1 Templates

In [136]:
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools: {tools}.
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:"
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```

Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""

In [137]:
human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

### 4.2 Promt

In [138]:
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", system_prompt),
    MessagesPlaceholder("chat_history", optional=True),
    ("human", human_prompt),
  ]
)

Adding the tool names, descriptions and arguments using a [partial prompt template](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/partial/). This allows the agent to access the information pertaining to each tool including the intended use cases. This also means we can add and remove tools without altering our entire prompt template.

In [139]:
prompt = prompt.partial(
  tools=render_text_description_and_args(list(tools)),
  tool_names=", ".join([t.name for t in tools]),
)

# 5. Agent's memory and chain

In [140]:
memory = ConversationBufferMemory()

chain = (RunnablePassthrough.assign(
  agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
  chat_history=lambda x: memory.chat_memory.messages,
  )
  | prompt | llm | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(agent=chain, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory)

# 6. Query and RAG response

### 6.1 general content

In [141]:
agent_executor.invoke({"input": 'What is the capital of France?'})



> Entering new AgentExecutor chain...


Question: What is the capital of France?
Thought: This is a factual question, not related to the New Zealand Strategic Goods List category 9 items.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The capital of France is Paris."
}
```
Observation

> Finished chain.


{'input': 'What is the capital of France?',
 'history': '',
 'output': 'The capital of France is Paris.'}

### 6.2 RAG content

In [142]:
agent_executor.invoke({"input": 'What is 9D002 item?'})



> Entering new AgentExecutor chain...


Question: What is the description of 9D002 item?
Thought: I need to retrieve the context for the 9D002 item from the New Zealand Strategic Goods List category 9.
Action:
```
{
  "action": "get_NZSGL_cat9_items_context",
  "action_input": "9D002"
}
```

Observation[Document(metadata={}, page_content='9D002: “Software”, not specified in 9D003 or 9D004, specially designed or modified forthe “production” of equipment specified by 9A001 to 9A121 or 9B.\n9D003: “Software” incorporating “technology” specified by 9E003.h. and used in“FADEC systems” for systems specified by 9A or equipment specified by 9B.'), Document(metadata={}, page_content='9D002: “Software”, not specified in 9D003 or 9D004, specially designed or modified forthe “production” of equipment specified by 9A001 to 9A121 or 9B.\n9D003: “Software” incorporating “technology” specified by 9E003.h. and used in“FADEC systems” for systems specified by 9A or equipment specified by 9B.'), Documen

{'input': 'What is 9D002 item?',
 'history': 'Human: What is the capital of France?\nAI: The capital of France is Paris.',
 'output': "9D002 is 'Software', not specified in 9D003 or 9D004, specially designed or modified for the 'production' of equipment specified by 9A001 to 9A121 or 9B."}